In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
# Codes URL des départements français
DEPARTEMENTS = {
    "Ain": "ld01", "Aisne": "ld02", "Allier": "ld03", "Alpes-de-Haute-Provence": "ld04",
    "Hautes-Alpes": "ld05", "Alpes-Maritimes": "ld06", "Ardèche": "ld07", "Ardennes": "ld08",
    "Ariège": "ld09", "Aube": "ld10", "Aude": "ld11", "Aveyron": "ld12", "Bouches-du-Rhône": "ld13",
    "Calvados": "ld14", "Cantal": "ld15", "Charente": "ld16", "Charente-Maritime": "ld17",
    "Cher": "ld18", "Corrèze": "ld19", "Corse-du-Sud": "ld2A", "Haute-Corse": "ld2B",
    "Côte-d'Or": "ld21", "Côtes-d'Armor": "ld22", "Creuse": "ld23", "Dordogne": "ld24",
    "Doubs": "ld25", "Drôme": "ld26", "Eure": "ld27", "Eure-et-Loir": "ld28", "Finistère": "ld29",
    "Gard": "ld30", "Haute-Garonne": "ld31", "Gers": "ld32", "Gironde": "ld33", "Hérault": "ld34",
    "Ille-et-Vilaine": "ld35", "Indre": "ld36", "Indre-et-Loire": "ld37", "Isère": "ld38",
    "Jura": "ld39", "Landes": "ld40", "Loir-et-Cher": "ld41", "Loire": "ld42",
    "Haute-Loire": "ld43", "Loire-Atlantique": "ld44", "Loiret": "ld45", "Lot": "ld46",
    "Lot-et-Garonne": "ld47", "Lozère": "ld48", "Maine-et-Loire": "ld49", "Manche": "ld50",
    "Marne": "ld51", "Haute-Marne": "ld52", "Mayenne": "ld53", "Meurthe-et-Moselle": "ld54",
    "Meuse": "ld55", "Morbihan": "ld56", "Moselle": "ld57", "Nièvre": "ld58", "Nord": "ld59",
    "Oise": "ld60", "Orne": "ld61", "Pas-de-Calais": "ld62", "Puy-de-Dôme": "ld63",
    "Pyrénées-Atlantiques": "ld64", "Hautes-Pyrénées": "ld65", "Pyrénées-Orientales": "ld66",
    "Bas-Rhin": "ld67", "Haut-Rhin": "ld68", "Rhône": "ld69", "Haute-Saône": "ld70",
    "Saône-et-Loire": "ld71", "Sarthe": "ld72", "Savoie": "ld73", "Haute-Savoie": "ld74",
    "Paris": "ld75", "Seine-Maritime": "ld76", "Seine-et-Marne": "ld77", "Yvelines": "ld78",
    "Deux-Sèvres": "ld79", "Somme": "ld80", "Tarn": "ld81", "Tarn-et-Garonne": "ld82",
    "Var": "ld83", "Vaucluse": "ld84", "Vendée": "ld85", "Vienne": "ld86", "Haute-Vienne": "ld87",
    "Vosges": "ld88", "Yonne": "ld89", "Territoire de Belfort": "ld90", "Essonne": "ld91",
    "Hauts-de-Seine": "ld92", "Seine-Saint-Denis": "ld93", "Val-de-Marne": "ld94",
    "Val-d'Oise": "ld95", "Guadeloupe": "ld971", "Martinique": "ld972", "Guyane": "ld973",
    "La Réunion": "ld974", "Mayotte": "ld976"
}

# Définition des filtres de biens (type d'annonce)
FILTRES_BIENS = {
    "Maison": "th",
    "Appartement": "tf"
}

# Nombre maximum de pages à extraire par département (limite du site)
NB_PAGES_MAX = 30

# Définition des filtres de superficie (code url, nom pour l'affichage)
# Nous utilisons les bornes données pour créer des intervalles consécutifs.
FILTRES_SUPERFICIE = {
    "s-20": "0m² à 20m²",     # Superficie Max 20
    "s20-25": "20m² à 25m²",  # Min 20, Max 25
    "s25-30": "25m² à 30m²",
    "s30-35": "30m² à 35m²",
    "s35-40": "35m² à 40m²",
    "s40-50": "40m² à 50m²",
    "s50-60": "50m² à 60m²",
    "s60-70": "60m² à 70m²",
    "s70-80": "70m² à 80m²",
    "s80-90": "80m² à 90m²",
    "s90-100": "90m² à 100m²",
    "s100-120": "100m² à 120m²",
    "s120-140": "120m² à 140m²",
    "s140-160": "140m² à 160m²",
    "s160-180": "160m² à 180m²",
    "s180-200": "180m² à 200m²",
    "s200-250": "200m² à 250m²",
    "s250": "Plus de 250m²" # Superficie Min 250 (sans max)
}

In [ ]:
# Récupération d'une page

def get_annonces_page(code_filtre, code_departement, code_superficie, page_num):
    """
    Retourne les liens trouvés sur une page filtrée par type de bien, département et superficie.
    Exemple d'URL : https://www.etreproprio.com/annonces/th.ld75.s20-25.odd.g2#list
    """
    # Construction du préfixe qui inclut tous les filtres (ex: th.ld75.s20-25)
    prefixe = f"{code_filtre}.{code_departement}.{code_superficie}"
    
    if page_num == 1:
        # URL pour la première page (ex: th.ld75.s20-25#list)
        url = f"https://www.etreproprio.com/annonces/{prefixe}#list"
    else:
        # URL pour la page N > 1 (ex: th.ld75.s20-25.odd.g2#list)
        url = f"https://www.etreproprio.com/annonces/{prefixe}.odd.g{page_num}#list"

    print(f"\n Chargement URL : {url}")

    try:
        r = requests.get(url)
        r.raise_for_status() # Lève une erreur pour les mauvaises réponses (4xx ou 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Erreur de requête pour l'URL {url}: {e}")
        return []

    soup = BeautifulSoup(r.content, "html.parser")

    zone = soup.find("div", class_="ep-search-list-wrapper")
    if not zone:
        # Ce message est normal si le filtre ne retourne aucune annonce
        return []

    links = []
    for a in zone.find_all("a", href=True):
        href = a["href"]
        if "https://www.etreproprio.com/immobilier-" in href:
            links.append(href)

    return links

In [ ]:
# Extraction des infos de l'annonce

def extract_info(url):
    """Extrait prix, surfaces (int/terrain), nb pièces, ville et code postal d’une annonce."""
    
    # Valeurs par défaut en cas d'erreur ou d'absence
    prix = "N/A"
    surface_interieure = "N/A"
    surface_terrain = "N/A"
    nb_pieces = "N/A"
    ville = "N/A"
    code_postal = "N/A"

    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de l'annonce {url}: {e}")
        return prix, surface_interieure, surface_terrain, nb_pieces, ville, code_postal

    # 1. Prix
    try:
        prix = soup.find("div", class_="ep-price").text.strip().replace(" ", "").replace("\xa0", "")
    except:
        pass

    # 2. Surface Intérieure (ep-area)
    # Correction : Conserver seulement la première valeur (avant le '/')
    try:
        text = soup.find("div", class_="ep-area").text.strip().replace(" ", "").replace("\xa0", "")
        surface_interieure = text.split('/')[0]
    except:
        pass

    # 3. Surface Terrain (dtl-main-surface-terrain)
    # Correction : Nettoyage du préfixe '/' s'il existe.
    try:
        surface_terrain_raw = soup.find("span", class_="dtl-main-surface-terrain").text.strip().replace(" ", "").replace("\xa0", "")
        if surface_terrain_raw.startswith('/'):
            surface_terrain = surface_terrain_raw[1:].strip()
        else:
            surface_terrain = surface_terrain_raw
    except:
        pass

    # 4. Nombre de pièces (ep-room)
    try:
        nb_pieces = soup.find("div", class_="ep-room").text.strip().replace("pièces", "").replace("pièce", "").strip()
    except:
        pass

    # 5 & 6. Ville et Code Postal (ep-loc)
    try:
        loc_text = soup.find("div", class_="ep-loc").text.strip()
        loc_text = loc_text.replace("—", "").replace("\xa0", " ").strip()
        
        parts = loc_text.split()
        
        if parts and parts[-1].isdigit():
            code_postal_raw = parts[-1]
            
            # CORRECTION : zfill(5) garantit 5 chiffres, ajoutant des zéros si nécessaire (ex: "1800" devient "01800")
            code_postal = code_postal_raw.zfill(5) 
            
            # Ville : tout ce qui précède le code postal
            ville = " ".join(parts[:-1]).strip()
        else:
            ville = loc_text
            code_postal = "N/A"
            
    except:
        pass

    return prix, surface_interieure, surface_terrain, nb_pieces, ville, code_postal

In [ ]:
# Fonction simplifiée pour vérifier l'existence des pages

def get_annonces_page_count(code_filtre, code_departement, code_superficie, page_num):
    """
    Vérifie si une page spécifique contient des annonces.
    Retourne le nombre d'annonces trouvées (0 si vide ou erreur).
    """
    # Construction de l'URL comme dans get_annonces_page
    prefixe = f"{code_filtre}.{code_departement}.{code_superficie}"
    
    if page_num == 1:
        url = f"https://www.etreproprio.com/annonces/{prefixe}#list"
    else:
        url = f"https://www.etreproprio.com/annonces/{prefixe}.odd.g{page_num}#list"

    try:
        r = requests.get(url, timeout=10) # Ajout d'un timeout pour les requêtes de vérification
        r.raise_for_status()
    except requests.exceptions.RequestException:
        return 0 # Retourne 0 si la requête échoue (page non trouvée, erreur 404/500, etc.)

    soup = BeautifulSoup(r.content, "html.parser")

    # Cherche la zone contenant les annonces
    zone = soup.find("div", class_="ep-search-list-wrapper")
    if not zone:
        return 0

    # Compte le nombre de liens d'annonces trouvés
    links_count = 0
    for a in zone.find_all("a", href=True):
        href = a["href"]
        if "https://www.etreproprio.com/immobilier-" in href:
            links_count += 1

    return links_count

In [ ]:
# =================================================================
# TRAITEMENT AVEC EXÉCUTION PARALLÈLE
# =================================================================

from concurrent.futures import ThreadPoolExecutor, as_completed

MAX_WORKERS = 8 # Nombre de requêtes d'extraction de détails lancées simultanément

total_annonces_global = 0  # Compteur des annonces uniques extraites
ALL_ANNONCES = []          # Liste des annonces uniques
PROCESSED_LINKS = set()    # Set pour stocker et vérifier les liens uniques

# Boucle 0 : Type de bien
for nom_bien, code_bien in FILTRES_BIENS.items():
    
    print("\n" + "#"*100)
    print(f"COMMENCEMENT DE L'EXTRACTION POUR LE TYPE DE BIEN : {nom_bien}")
    print("#"*100)

    # Boucle 1 : Départements
    for nom_departement, code_departement in DEPARTEMENTS.items():
        
        print("\n" + "="*80)
        print(f"      DÉPARTEMENT EN COURS : {nom_departement} ({code_departement})")
        print("="*80)

        # Boucle 2 : Superficies
        for code_superficie, nom_superficie in FILTRES_SUPERFICIE.items():
            
            print("\n" + "-"*50)
            print(f"   FILTRE SUPERFICIE : {nom_superficie} ({code_superficie})")
            print("-"*50)

            # 1. PHASE DE COLLECTE DES LIENS (SÉQUENTIELLE)
            links_to_extract_details = []
            
            for page_num in range(1, NB_PAGES_MAX + 1):

                liens_page = get_annonces_page(code_bien, code_departement, code_superficie, page_num)

                if not liens_page:
                    print(f"\n[INFO] Arrêt : Aucune annonce trouvée sur la Page {page_num}. Fin de la pagination.")
                    break
                
                print(f"Page {page_num} ({len(liens_page)} liens trouvés)")

                # Dédoublonnage et ajout des liens uniques à la liste de traitement parallèle
                for lien in liens_page:
                    if lien not in PROCESSED_LINKS:
                        PROCESSED_LINKS.add(lien)
                        links_to_extract_details.append(lien)
            
            # 2. PHASE D'EXTRACTION DES DÉTAILS (PARALLÈLE)
            
            nb_liens_uniques = len(links_to_extract_details)
            if nb_liens_uniques > 0:
                print(f"   [INFO] Lancement de l'extraction parallèle ({MAX_WORKERS} workers) pour {nb_liens_uniques} annonces uniques...")
                
                with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                    
                    # Soumettre la tâche extract_info pour chaque lien
                    future_to_url = {executor.submit(extract_info, link): link for link in links_to_extract_details}
                    
                    for future in as_completed(future_to_url):
                        lien = future_to_url[future]
                        try:
                            # Récupérer les résultats du thread
                            prix, surface_interieure, surface_terrain, nb_pieces, ville, code_postal = future.result()
                            
                            annonce_data = {
                                "Type de Bien": nom_bien,
                                "Departement": nom_departement,
                                "Ville": ville,
                                "Code_Postal": code_postal,
                                "Nb_Pieces": nb_pieces,
                                "Surface_Interieure": surface_interieure,
                                "Surface_Terrain": surface_terrain,
                                "Prix": prix,
                                "Lien": lien
                            }
                            
                            ALL_ANNONCES.append(annonce_data)
                            
                        except Exception as exc:
                            # Gérer les erreurs de connexion/timeout dans les threads
                            print(f'   [ERREUR PARALLÈLE] Le lien {lien} a généré une exception: {exc}')

            print(f"\n--- {nb_liens_uniques} ANNONCES UNIQUES TRAITÉES POUR LE FILTRE {nom_superficie} ---")
            
# Mettre à jour le compteur global
total_annonces_global = len(ALL_ANNONCES)


# Bloc de Sauvegarde CSV (Inchangé dans sa logique)

print("\n\n" + "#"*80)
print(f"### FIN DE L'EXTRACTION - SAUVEGARDE DES DONNÉES UNIQUES ###")
print("#"*80)

if ALL_ANNONCES:
    
    fieldnames = list(ALL_ANNONCES[0].keys())
    output_filename = "annonces_etreproprio_maisons_appartements.csv"
    
    try:
        with open(output_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')

            writer.writeheader()
            writer.writerows(ALL_ANNONCES)

        print(f"\n[SUCCÈS] Toutes les données ont été sauvegardées dans le fichier : {output_filename}")
        print(f"Nombre total d'annonces uniques extraites : {total_annonces_global}")

    except Exception as e:
        print(f"\n[ERREUR] Impossible de sauvegarder le fichier CSV : {e}")
else:
    print("\n[INFO] Aucune donnée à sauvegarder.")

In [ ]:
# Traitement

total_annonces_global = 0 # Nombre total de liens trouvés (y compris doublons)
ALL_ANNONCES = [] # Liste des annonces uniques
PROCESSED_LINKS = set() # Set pour stocker et vérifier les liens uniques

# Boucle 0 : Type de bien
for nom_bien, code_bien in FILTRES_BIENS.items():
    
    print("\n" + "#"*100)
    print(f"COMMENCEMENT DE L'EXTRACTION POUR LE TYPE DE BIEN : {nom_bien}")
    print("#"*100)

    # Boucle 1 : Départements
    for nom_departement, code_departement in DEPARTEMENTS.items():
        
        print("\n" + "="*80)
        print(f"      DÉPARTEMENT EN COURS : {nom_departement} ({code_departement})")
        print("="*80)

        # Boucle 2 : Superficies
        for code_superficie, nom_superficie in FILTRES_SUPERFICIE.items():
            
            print("\n" + "-"*50)
            print(f"   FILTRE SUPERFICIE : {nom_superficie} ({code_superficie})")
            print("-"*50)

            total_annonces_filtre = 0
            
            # Boucle 3 : Pages (1 à 30)
            for page_num in range(1, NB_PAGES_MAX + 1):

                liens = get_annonces_page(code_bien, code_departement, code_superficie, page_num)

                if not liens:
                    print(f"\n[INFO] Arrêt : Aucune annonce trouvée sur la Page {page_num}. Fin de la pagination pour ce filtre.")
                    break
                
                print(f"Page {page_num} ({len(liens)} annonces trouvées)")
                total_annonces_filtre += len(liens)
                total_annonces_global += len(liens) # Compte le nombre de liens trouvés (brut)

                # Dédoublonnage
                for lien in liens:
                    
                    if lien in PROCESSED_LINKS:
                        # Si le lien est déjà dans l'ensemble, on l'ignore (doublon)
                        print(f"   [SKIP] Doublon détecté, lien ignoré: {lien}")
                        continue
                    
                    # Le lien est unique, on l'ajoute à l'ensemble pour le marquer comme traité
                    PROCESSED_LINKS.add(lien)
                    
                    # Récupération de toutes les infos
                    prix, surface_interieure, surface_terrain, nb_pieces, ville, code_postal = extract_info(lien)
                    
                    # 1. Création d'un dictionnaire pour cette annonce
                    annonce_data = {
                        "Type de Bien": nom_bien,
                        "Departement": nom_departement,
                        "Ville": ville,
                        "Code_Postal": code_postal,
                        "Nb_Pieces": nb_pieces,
                        "Surface_Interieure": surface_interieure,
                        "Surface_Terrain": surface_terrain,
                        "Prix": prix,
                        "Lien": lien
                    }
                    
                    # 2. Ajout à la liste globale
                    ALL_ANNONCES.append(annonce_data)
                    
                    print(f"   > [{nom_bien} - {ville}] Prix: {prix}")

            print(f"\n--- TOTAL ANNONCES {nom_bien} / {nom_departement} ({nom_superficie}): {total_annonces_filtre} ---")


# Bloc de Sauvegarde CSV
# Ce bloc utilise la liste ALL_ANNONCES qui ne contient maintenant que les liens uniques

print("\n\n" + "#"*80)
print(f"### FIN DE L'EXTRACTION - SAUVEGARDE DES DONNÉES UNIQUES ###")
print("#"*80)

if ALL_ANNONCES:
    
    fieldnames = list(ALL_ANNONCES[0].keys())
    output_filename = "annonces_etreproprio_maisons_appartements.csv"
    
    try:
        with open(output_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')

            writer.writeheader()
            writer.writerows(ALL_ANNONCES)

        print(f"\n[SUCCÈS] Toutes les données ont été sauvegardées dans le fichier : {output_filename}")
        print(f"Nombre total d'annonces uniques extraites : {len(ALL_ANNONCES)}")
        print(f"Nombre de liens trouvés (brut, avant dédoublonnage) : {total_annonces_global}") # Pour comparaison

    except Exception as e:
        print(f"\n[ERREUR] Impossible de sauvegarder le fichier CSV : {e}")
else:
    print("\n[INFO] Aucune donnée à sauvegarder.")